In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import request

In [19]:
page = 1

comp_list=[]
date_list=[]


total_page = 233

while True:
    url = 'https://finance.naver.com/research/debenture_list.nhn?' 
    
    
    params = {
        'page' : page
    }

    
    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.content, 'html.parser')

    #링크
    td_link = soup.find('table', class_='type_1').find_all('td', class_='file')
    td_date = soup.find('table', class_='type_1').find_all('td', class_='date')
    tr_comp = soup.find('table', class_= 'type_1').find_all('tr')

    #증권사 이름 추출
    
    for tr in tr_comp:
        try:
            comp=tr.find_all('td')[1].text
            comp_list.append(comp)
            print(f'{comp}', end = ' ')
        except:
            continue
    
    #날짜 추출
    
    for idx, td in enumerate(td_date):
        if idx % 2 == 0:
            date_list.append(td.text)
            print(f'{td.text} 읽음', end=' ')

    
        
    if page>=total_page : 
        break
        
    page+=1 
    print()

한화투자증권 교보증권 교보증권 하나증권 다올투자증권 유진투자증권 신한투자증권 키움증권 유안타증권 하이투자증권 IBK투자증권 메리츠증권 교보증권 하나증권 하나증권 한화투자증권 다올투자증권 유진투자증권 키움증권 유안타증권 신한투자증권 신한투자증권 교보증권 다올투자증권 유진투자증권 유안타증권 신한투자증권 한화투자증권 한화투자증권 교보증권 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.21 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.20 읽음 24.03.19 읽음 24.03.19 읽음 24.03.19 읽음 24.03.19 읽음 24.03.19 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 
신한투자증권 다올투자증권 하이투자증권 유안타증권 키움증권 유진투자증권 IBK투자증권 신한투자증권 신한투자증권 한화투자증권 교보증권 다올투자증권 유진투자증권 유안타증권 키움증권 교보증권 다올투자증권 유안타증권 유진투자증권 메리츠증권 신한투자증권 교보증권 하나증권 하나증권 유안타증권 유진투자증권 다올투자증권 신한투자증권 유안타증권 교보증권 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.18 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.15 읽음 24.03.14 읽음 24.03.14 읽음 24.03.14 읽음 24.03.14 읽음 24.03.14 읽음 24.03.13 읽음 24.03.13 읽음

In [3]:
!pip install pdfminer.six

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
    --------------------------------------- 0.1/5.6 MB 3.6 MB/s eta 0:00:02
   ------------ --------------------------- 1.8/5.6 MB 22.3 MB/s eta 0:00:01
   ----------------------------- ---------- 4.2/5.6 MB 33.4 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 35.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------------------------------------- -- 2.7/2.9 MB 58.1 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 46.5 MB/s eta 0:00:00


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [9]:
def pdfToText(fname):

    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'rawReport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages, password = password,
                                      caching = caching, check_extractable = True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)

In [10]:
len(comp_list)

6970

In [11]:
len(date_list)

6970

In [25]:
if len(comp_list) != len(date_list):
    raise ValueError("comp_list와 date_list의 길이가 다릅니다.")


In [26]:
import pandas as pd

text_list=[]
for idx, comp in enumerate(comp_list):
    
    page = (idx // 30)+1
    p_no = (idx % 30)+1
    
    text=pdfToText(f'{page}_{p_no}.pdf')
    
    print(f'{page}_{p_no}.pdf 텍스트 추출...')
    text=text.replace('\n','')
    text=text.replace('\r','')
    text=text.replace('\t','')
    
    text_list.append(text)
    
    if (idx+1) % 30 == 0:
        dict_report ={
            '증권사' : comp_list[idx-29:idx+1],
            '날짜' : date_list[idx-29:idx+1],
            '텍스트' : text_list
        }
        
        df = pd.DataFrame(dict_report)
        df.to_csv(f'bond_report page_{page}.csv', index=False)
        
        text_list = []


rest = len(text_list)

dict_report ={
            '증권사' : comp_list[len(comp_list)-rest:],
            '날짜' : date_list[len(date_list)-rest:],
            '텍스트' : text_list
            }
        
df = pd.DataFrame(dict_report)
df.to_csv(f'bond_report page_{page}.csv', index=False)

1_1.pdf 텍스트 추출...
1_2.pdf 텍스트 추출...
1_3.pdf 텍스트 추출...
1_4.pdf 텍스트 추출...
1_5.pdf 텍스트 추출...
1_6.pdf 텍스트 추출...
1_7.pdf 텍스트 추출...
1_8.pdf 텍스트 추출...
1_9.pdf 텍스트 추출...
1_10.pdf 텍스트 추출...
1_11.pdf 텍스트 추출...
1_12.pdf 텍스트 추출...
1_13.pdf 텍스트 추출...
1_14.pdf 텍스트 추출...
1_15.pdf 텍스트 추출...
1_16.pdf 텍스트 추출...
1_17.pdf 텍스트 추출...
1_18.pdf 텍스트 추출...
1_19.pdf 텍스트 추출...
1_20.pdf 텍스트 추출...
1_21.pdf 텍스트 추출...
1_22.pdf 텍스트 추출...
1_23.pdf 텍스트 추출...
1_24.pdf 텍스트 추출...
1_25.pdf 텍스트 추출...
1_26.pdf 텍스트 추출...
1_27.pdf 텍스트 추출...
1_28.pdf 텍스트 추출...
1_29.pdf 텍스트 추출...
1_30.pdf 텍스트 추출...
2_1.pdf 텍스트 추출...
2_2.pdf 텍스트 추출...
2_3.pdf 텍스트 추출...
2_4.pdf 텍스트 추출...
2_5.pdf 텍스트 추출...
2_6.pdf 텍스트 추출...
2_7.pdf 텍스트 추출...
2_8.pdf 텍스트 추출...
2_9.pdf 텍스트 추출...
2_10.pdf 텍스트 추출...
2_11.pdf 텍스트 추출...
2_12.pdf 텍스트 추출...
2_13.pdf 텍스트 추출...
2_14.pdf 텍스트 추출...
2_15.pdf 텍스트 추출...
2_16.pdf 텍스트 추출...
2_17.pdf 텍스트 추출...
2_18.pdf 텍스트 추출...
2_19.pdf 텍스트 추출...
2_20.pdf 텍스트 추출...
2_21.pdf 텍스트 추출...
2_22.pdf 텍스트 추출...
2_23.pdf 텍스트 추출...
2_24.pdf 텍스

In [27]:
import pandas as pd

def process_and_save_data(comp_list, date_list, text_list, page):
    dict_report ={
        '증권사': comp_list,
        '날짜': date_list,
        '텍스트': text_list
    }
    df = pd.DataFrame(dict_report)
    df.to_csv(f'bond_report_page_{page}.csv', index=False)

comp_per_page = 30
total_page = (len(comp_list) + comp_per_page - 1) // comp_per_page

for page in range(1, total_page + 1):
    comp_slice = comp_list[(page - 1) * comp_per_page: page * comp_per_page]
    date_slice = date_list[(page - 1) * comp_per_page: page * comp_per_page]
    text_slice = []
    for idx, comp in enumerate(comp_slice):
        p_no = (page - 1) * comp_per_page + idx + 1
        text = pdfToText(f'{page}_{p_no}.pdf')
        text = text.replace('\n', '').replace('\r', '').replace('\t', '')
        text_slice.append(text)

    process_and_save_data(comp_slice, date_slice, text_slice, page)



In [29]:
import pandas as pd
import fitz  # PyMuPDF
from concurrent.futures import ThreadPoolExecutor, as_completed

def pdfToText(filepath):
    try:
        doc = fitz.open(filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {filepath}, 오류: {e}")
        return None

def process_and_save_data(comp_list, date_list, text_list, page):
    dict_report = {
        '증권사': comp_list,
        '날짜': date_list,
        '텍스트': text_list
    }
    df = pd.DataFrame(dict_report)
    df.to_csv(f'bond_report_page_{page}.csv', index=False)

def process_page(page, comp_per_page):
    comp_slice = comp_list[(page - 1) * comp_per_page: page * comp_per_page]
    date_slice = date_list[(page - 1) * comp_per_page: page * comp_per_page]
    text_slice = []
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_pdf = {executor.submit(pdfToText, f'{page}_{(page - 1) * comp_per_page + idx + 1}.pdf'): idx for idx, _ in enumerate(comp_slice)}
        for future in as_completed(future_to_pdf):
            idx = future_to_pdf[future]
            text = future.result()
            if text:
                text_slice.append(text.replace('\n', '').replace('\r', '').replace('\t', ''))
            else:
                # 실패한 경우 빈 문자열로 대체
                text_slice.append("")

    process_and_save_data(comp_slice, date_slice, text_slice, page)

if __name__ == "__main__":
    # 예시: comp_list와 date_list가 이미 정의되어 있다고 가정
    comp_list = ["증권사A", "증권사B", "증권사C"]  # 실제 사용 시 적절한 값으로 대체
    date_list = ["2024-03-21", "2024-03-22", "2024-03-23"]  # 실제 사용 시 적절한 값으로 대체
    
    comp_per_page = 30
    total_page = (len(comp_list) + comp_per_page - 1) // comp_per_page
    
    for page in range(1, total_page + 1):
        process_page(page, comp_per_page)


파일 처리 중 오류 발생: 1_1.pdf, 오류: no such file: '1_1.pdf'
파일 처리 중 오류 발생: 1_2.pdf, 오류: no such file: '1_2.pdf'
파일 처리 중 오류 발생: 1_3.pdf, 오류: no such file: '1_3.pdf'
